In [1]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import random

In [2]:
num_rows = 1000  # Número de filas

# Generar valores aleatorios para las columnas 'src' y 'dst'
src_values = [random.choice(["A", "B"]) for _ in range(num_rows)]
dst_values = [random.choice(["B", "C"]) for _ in range(num_rows)]

# Crear el DataFrame
df = pd.DataFrame({"src": src_values, "dst": dst_values})

In [3]:
A = list(df["src"].unique())
B = list(df["dst"].unique())
node_list = set(A + B)

In [61]:
G = nx.MultiGraph()
for i in node_list:
    G.add_node(i)

In [62]:
for i, j in df.iterrows():
    G.add_edges_from([(j["src"], j["dst"])])

In [63]:
pos = nx.spring_layout(G, k=0.5, iterations=50)
for n, p in pos.items():
    G.nodes[n]["pos"] = p

In [64]:
G.edges()

MultiEdgeDataView([('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'), ('A', 'B'

In [66]:
edge_counts = {(edge[0], edge[1]): len(G.edges(edge[0], edge[1])) for edge in G.edges()}
max_degree = max(edge_counts.values())
edge_colors = [(((edge / max_degree) * 5) ** 2) for edge in edge_counts.values()]

traces = []
# Actualizar el grosor de las aristas y añadir las líneas a edge_trace
for i, ((src, dst), count) in enumerate(edge_counts.items()):
    traces.append(
        go.Scatter(
            x=[pos[src][0], pos[dst][0]],
            y=[pos[src][1], pos[dst][1]],
            hoverinfo="none",
            mode="lines",
            line=dict(width=edge_colors[i]),
        )
    )
edge_trace = traces

In [67]:
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode="markers",
    hoverinfo="text",
    marker=dict(
        showscale=True,
        colorscale="RdBu",
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10, title="Node Connections", xanchor="left", titleside="right"
        ),
        line=dict(width=0),
    ),
)

for node in G.nodes():
    x, y = G.nodes[node]["pos"]
    node_trace["x"] += tuple([x])
    node_trace["y"] += tuple([y])

In [68]:
G.degree

MultiDegreeView({'A': 498, 'C': 512, 'B': 990})

In [69]:
for i, node in enumerate(G.nodes()):
    node_trace["marker"]["color"] += tuple([G.degree[node]])
    node_info = node + " # of connections: " + str(G.degree[node])
    node_trace["text"] += tuple([node_info])

In [70]:
fig = go.Figure(
    data=[*edge_trace, node_trace],
    layout=go.Layout(
        title="<br>AT&T network connections",
        titlefont=dict(size=16),
        showlegend=False,
        hovermode="closest",
        margin=dict(b=20, l=5, r=5, t=40),
        annotations=[
            dict(text="No. of connections", showarrow=False, xref="paper", yref="paper")
        ],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    ),
)

fig.show()